In [55]:
#%pip install gradio
#%pip install PyPDF2

In [56]:
import gradio as gr

In [57]:
import re
import pickle
import pandas as pd
import PyPDF2

In [62]:
#Loading models
clf = pickle.load(open('clf.pkl','rb'))
rgr = pickle.load(open('rgr.pkl','rb'))
tfidf = pickle.load(open('tfidf.pkl','rb'))


def read_file(resume_text):
  pdfFileObject = open(resume_text, 'rb')
  pdfReader = PyPDF2.PdfReader(pdfFileObject)
  count = len(pdfReader.pages)
  for i in range(count):
    page = pdfReader.pages[i]
    output=[]
    output.append(page.extract_text())
  return output[0]

def clean_resume(resume_text):
    #URls: start with http, then matches one or more non-white space characters
    cleanTxt = re.sub('http\\S+',' ', resume_text)  
    #Hashtags: start with #, then matches one or more non-white space characters
    cleanTxt = re.sub('#\\S+',' ', cleanTxt)
    #Mentions: start with #, then matches one or more non-white space characters
    cleanTxt = re.sub('@\\S+',' ', cleanTxt)
    #Remove punctuation marks and other special characters from the text
    cleanTxt = re.sub(r'[^\w\s]', ' ', cleanTxt)
    #Remove any non-ASCII characters 
    cleanTxt = re.sub(r'[^\x00-\x7f]',' ', cleanTxt)    
    #Substitute 1 or more than 1 white space with 1 space
    cleanTxt = re.sub('\\s+',' ', cleanTxt)
    return cleanTxt

def predict_category(resume_text):
  read_resume = read_file(resume_text)
  cleaned_resume = clean_resume(read_resume)
  input_features = tfidf.transform([cleaned_resume])
  prediction = clf.predict(input_features)[0]
  predict_label = rgr.predict_proba(input_features)[:,1][0]
  predict_label_percentage = predict_label * 100
  return prediction, f"{predict_label_percentage:.2f}%"

interface = gr.Interface(
    fn=predict_category,
    inputs="file",
    outputs=[
        gr.Textbox(label="Predicted Category"),
        gr.Textbox(label="Probability of Success")  # Format probability to 2 decimal places
    ],
    title="Resume Screening App",
    description="Upload your resume to get a category prediction and success probability prediction.",
)

interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7875
* Running on public URL: https://7cfa24f486fa260a40.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
